<a href="https://colab.research.google.com/github/AtulAravindDas/DeepLearningAI_CrewAI_Agents/blob/main/Research_Article_Writer_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import sys

sys.path.append('/content/drive/MyDrive/')


In [ ]:
import os
from utils import *
import getpass

api_key=os.environ.get("OPENAI_API_KEY")

if api_key:
  openai_api_key=api_key
  os.environ["OPENAI_MODEL_NAME"]='gpt-4o'
  print("OpenAI API Key already set")
else:
  api_key=getpass.getpass("OpenAI API Key: ")
  openai_api_key=api_key
  os.environ["OPENAI_API_KEY"]=openai_api_key
  os.environ["OPENAI_MODEL_NAME"]='gpt-4o'
  print("OpenAI API Key set")

OpenAI API Key already set


In [ ]:
from IPython.display import clear_output

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan=Task(description=("1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
            "2. Identify the target audience, considering "
            "their interests and pain points.\n"
            "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,)


### Task: Write

In [ ]:
write=Task(description=("1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [ ]:
from IPython.display import Markdown
topic = "FIFA World Cup 2022"
result = crew.kickoff(inputs={"topic": topic})
clear_output(wait=True)
Markdown(result)

```markdown
# FIFA World Cup 2022: A Global Spectacle in Qatar

The FIFA World Cup 2022 is more than just a tournament; it is a cultural phenomenon that unites millions around the globe. Held in Qatar, this edition of the World Cup marks a departure from tradition, being the first played in winter due to the extreme summer heat of the Middle East. This shift in schedule has piqued the interest of football enthusiasts worldwide, eager to see how the event unfolds. As one of the most anticipated sporting events, the World Cup promises excitement, drama, and unforgettable moments on and off the field.

## Latest Trends and News: The Unique Qatar World Cup

The Qatar World Cup stands out for its groundbreaking innovations and unique challenges. For the first time, state-of-the-art stadiums have been constructed using sustainable practices, showcasing cutting-edge architectural designs. However, the journey to this tournament has not been without controversy, including concerns over human rights and the environmental impact of building new infrastructure. Despite these issues, Qatar is poised to deliver a memorable World Cup experience, blending modernity with traditional cultural elements.

## Key Players to Watch: Stars of the World Stage

Every World Cup brings a fresh wave of talent, and 2022 is no exception. Legendary figures like Lionel Messi and Cristiano Ronaldo are likely making their final World Cup appearances, adding an emotional layer to their performances. Meanwhile, emerging stars such as Kylian Mbappe are set to make their mark, with the potential to sway the outcomes of crucial matches. This year's tournament will be a blend of experience and youthful exuberance, a testament to the evolving landscape of international football.

## Teams and Strategies: A Tactical Battle

The FIFA World Cup 2022 will see a fierce competition among the world's top teams, each bringing unique tactical approaches. Traditional powerhouses like Brazil and Germany are expected to showcase their strategic prowess, while teams like Belgium and Croatia continue to prove their mettle on the international stage. Underdog teams, often overlooked, are poised to surprise audiences with innovative strategies and unyielding determination, reminding us that in football, anything is possible.

## Match Highlights and Results: Moments of Glory

As the tournament progresses, fans can expect a series of thrilling matches that will be etched in history. The group stages are critical, with each team vying for a coveted spot in the knockout rounds. Statistical analyses of player performances and team standings will provide deeper insights into the unfolding drama. From last-minute goals to stunning saves, the World Cup is a theater of dreams where heroes are made and legends are born.

## Audience Engagement: The Digital Fan Experience

In this digital age, fan engagement has reached new heights. Social media platforms like Twitter and Instagram are buzzing with real-time updates and fan interactions, creating a global conversation around the World Cup. Advanced technology is enhancing the viewer experience, with virtual reality and augmented reality offering fans new ways to experience matches. This digital transformation is not just about watching the game; it's about being part of a community, celebrating the joy of football together.

## Join the Conversation

As the FIFA World Cup 2022 unfolds, we invite you to share your thoughts and predictions in the comments section below. Subscribe to our blog for ongoing updates and analysis throughout the tournament. Let's celebrate the beautiful game and the stories that make this World Cup unforgettable.

---

Sources for this article include the [Official FIFA Website](https://www.fifa.com), as well as trusted sports news outlets like ESPN, BBC Sport, and Sky Sports. Follow the journey on social media for real-time updates and fan interactions.
```

This blog post is now well-aligned with journalistic standards and the brand's voice, providing balanced viewpoints and avoiding controversial topics. It is ready for publication.